<a href="https://colab.research.google.com/github/AbdulSheffa/IRP/blob/main/FYP_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix.git
%cd pytorch-CycleGAN-and-pix2pix


Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 21.70 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.
/content/pytorch-CycleGAN-and-pix2pix


In [3]:
!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.6 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408196 sha

In [5]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
from PIL import Image
from models.networks import ResnetGenerator, NLayerDiscriminator

# ------------------------------
# 1. Dataset with Self-Supervised Learning & Masking
# ------------------------------
class RainDatasetSSL(Dataset):
    def __init__(self, dataset_name, input_dir, target_dir, transform=None, sample_size=None, output_dir=None, save_masked=False, epoch=0):
        self.dataset_name = dataset_name
        self.input_dir = input_dir
        self.target_dir = target_dir
        self.transform = transform
        self.output_dir = output_dir
        self.save_masked = save_masked
        self.epoch = epoch

        self.input_images = sorted(os.listdir(input_dir))
        self.target_images = sorted(os.listdir(target_dir))

        if sample_size:
            indices = random.sample(range(len(self.input_images)), sample_size)
            self.input_images = [self.input_images[i] for i in indices]
            self.target_images = [self.target_images[i] for i in indices]

        if save_masked and output_dir:
            os.makedirs(output_dir, exist_ok=True)
            self.masked_images_to_save = random.sample(self.input_images, min(10, len(self.input_images)))

    def __len__(self):
        return len(self.input_images)

    def mask_image(self, img_tensor, img_name):
        """Apply dynamic masking for self-supervised learning & save masked image if required."""
        if img_tensor.dim() == 4:  # If batch, process each image separately
            return torch.stack([self.mask_image(single_img, img_name) for single_img in img_tensor])

        if img_tensor.dim() != 3:
            raise ValueError(f"Unexpected tensor shape {img_tensor.shape}, expected (C, H, W)")

        _, h, w = img_tensor.shape
        mask_size = (random.randint(h // 6, h // 4), random.randint(w // 6, w // 4))
        x, y = random.randint(0, h - mask_size[0]), random.randint(0, w - mask_size[1])

        masked_tensor = img_tensor.clone()
        masked_tensor[:, x:x + mask_size[0], y:y + mask_size[1]] = -1

        # ✅ Ensure the output directory exists before saving
        if self.save_masked and self.output_dir:
            os.makedirs(self.output_dir, exist_ok=True)  # ✅ Ensure the directory exists

            save_path = os.path.join(self.output_dir, f"masked_{self.dataset_name}_epoch_{self.epoch}_{img_name}")
            save_image((masked_tensor + 1) / 2, save_path)  # Denormalizing before saving
            # print(f"✅ Saved masked image: {save_path}")

        return masked_tensor


    def __getitem__(self, idx):
        input_img_path = os.path.join(self.input_dir, self.input_images[idx])
        target_img_path = os.path.join(self.target_dir, self.target_images[idx])

        input_image = Image.open(input_img_path).convert("RGB")
        target_image = Image.open(target_img_path).convert("RGB")

        if self.transform:
            input_image = self.transform(input_image)
            target_image = self.transform(target_image)

        masked_image = self.mask_image(input_image.clone(), self.input_images[idx])
        return masked_image, input_image, target_image, self.input_images[idx]  # ✅ Ensure 4 elements

# ------------------------------
# 2. Masked GAN-Based DeRain CycleGAN (Without Self-Attention)
# ------------------------------
class DeRainCycleGANSSL:
    def __init__(self, input_nc, output_nc, ngf, ndf, device):
        self.device = device
        self.netG_A = ResnetGenerator(input_nc, output_nc, ngf, norm_layer=nn.InstanceNorm2d ,n_blocks=9).to(device)
        self.netG_B = ResnetGenerator(output_nc, input_nc, ngf).to(device)
        self.netD_A = NLayerDiscriminator(output_nc, ndf).to(device)
        self.netD_B = NLayerDiscriminator(input_nc, ndf).to(device)

        self.criterionGAN = nn.MSELoss()
        self.criterionCycle = nn.L1Loss()
        self.criterionPerceptual = nn.MSELoss()

        self.vgg19 = models.vgg19(pretrained=True).features[:36].eval().to(device)
        for param in self.vgg19.parameters():
            param.requires_grad = False

        self.optimizer_G = torch.optim.Adam(
            list(self.netG_A.parameters()) + list(self.netG_B.parameters()), lr=0.00005, betas=(0.5, 0.999)
        )
        self.optimizer_D = torch.optim.Adam(
            list(self.netD_A.parameters()) + list(self.netD_B.parameters()), lr=0.00001, betas=(0.5, 0.999)
        )

    def perceptual_loss(self, x, y):
        return self.criterionPerceptual(self.vgg19(x), self.vgg19(y))

    def forward(self, real_A, real_B):
        fake_B = self.netG_A(real_A)
        rec_A = self.netG_B(fake_B)
        fake_A = self.netG_B(real_B)
        rec_B = self.netG_A(fake_A)
        return fake_B, rec_A, fake_A, rec_B

    def optimize_parameters(self, real_A, real_B, scaler):
        """Optimizes generator and discriminator parameters for one iteration."""

        self.optimizer_G.zero_grad()
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            real_A, real_B = real_A.half(), real_B.half()
            fake_B, rec_A, fake_A, rec_B = self.forward(real_A, real_B)
            loss_G_A = self.criterionGAN(self.netD_A(fake_B), torch.ones_like(self.netD_A(fake_B)))
            loss_G_B = self.criterionGAN(self.netD_B(fake_A), torch.ones_like(self.netD_B(fake_A)))
            loss_cycle_A = self.criterionCycle(rec_A, real_A) * 10.0
            loss_cycle_B = self.criterionCycle(rec_B, real_B) * 10.0
            loss_perceptual = self.perceptual_loss(fake_B, real_B) * 0.2  # Reduced from 0.4
            loss_L1 = self.criterionCycle(fake_B, real_B) * 10.0  # Introduced L1 loss
            loss_G = 2.0 * loss_G_A + 2.0 * loss_G_B + 4.0 * loss_cycle_A + 4.0 * loss_cycle_B + 0.5 * loss_perceptual + loss_L1

        scaler.scale(loss_G).backward()
        scaler.step(self.optimizer_G)
        scaler.update()

        self.optimizer_D.zero_grad()
        with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
            loss_D_A = self.criterionGAN(self.netD_A(real_B.half()), torch.ones_like(self.netD_A(real_B.half())))
            loss_D_B = self.criterionGAN(self.netD_B(real_A.half()), torch.ones_like(self.netD_B(real_A.half())))
            loss_D = (loss_D_A + loss_D_B) * 0.3

        scaler.scale(loss_D).backward()
        scaler.step(self.optimizer_D)
        scaler.update()

        return loss_G.detach(), loss_D_A.detach(), loss_D_B.detach()


In [ ]:
import os
import torch
import torch.nn as nn
import torch.cuda.amp as amp  # ✅ Import amp explicitly
from torchvision import transforms

# ✅ Initialize Mixed Precision Training
scaler = amp.GradScaler()

# ✅ Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Define Checkpoint Directory
checkpoint_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint/"
os.makedirs(checkpoint_dir, exist_ok=True)  # Ensure checkpoint folder exists

# ✅ Initialize Model BEFORE Loading Checkpoint
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=128, ndf=256, device=device)

# ✅ Function to Load the Model from Checkpoint
def load_model_only(model, checkpoint_dir):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pth")]
    if not checkpoints:
        print("⚠️ No checkpoint found. Model will remain uninitialized.")
        return None

    # ✅ Load the latest checkpoint
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("_")[-1].split(".")[0]))[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)

    print(f"🔄 Loading model from {checkpoint_path}...")

    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.netG_A.load_state_dict(checkpoint['model_G_A_state_dict'])
    model.netG_B.load_state_dict(checkpoint['model_G_B_state_dict'])
    model.netD_A.load_state_dict(checkpoint['model_D_A_state_dict'])
    model.netD_B.load_state_dict(checkpoint['model_D_B_state_dict'])

    print("✅ Model Loaded Successfully!")

# ✅ Load the Model from Checkpoint (Without Training)
load_model_only(model, checkpoint_dir)


<ipython-input-6-1127d71797bc>:8: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:03<00:00, 168MB/s]


🔄 Loading model from /content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint/checkpoint_epoch_79.pth...


<ipython-input-6-1127d71797bc>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device)


✅ Model Loaded Successfully!


In [ ]:
import os
import torch

def load_latest_checkpoint(model, optimizer_G, optimizer_D, scheduler_G, scheduler_D, scaler, checkpoint_dir, device):
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith(".pth")]
    if not checkpoints:
        print("⚠️ No checkpoint found, starting from scratch.")
        return 0  # Start from epoch 0

    # Sort and get the latest checkpoint
    latest_checkpoint = sorted(checkpoints, key=lambda x: int(x.split("_")[-1].split(".")[0]))[-1]
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint)

    print(f"🔄 Resuming training from {checkpoint_path}")

    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device)
    model.netG_A.load_state_dict(checkpoint['model_G_A_state_dict'])
    model.netG_B.load_state_dict(checkpoint['model_G_B_state_dict'])
    model.netD_A.load_state_dict(checkpoint['model_D_A_state_dict'])
    model.netD_B.load_state_dict(checkpoint['model_D_B_state_dict'])
    optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
    optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    scheduler_G.load_state_dict(checkpoint['scheduler_G_state_dict'])
    scheduler_D.load_state_dict(checkpoint['scheduler_D_state_dict'])
    scaler.load_state_dict(checkpoint['scaler_state_dict'])

    return checkpoint['epoch'] + 1  # Resume from the next epoch


In [ ]:
import os
import torch
import torch.nn as nn
import torch.cuda.amp as amp  # Mixed Precision Training
from torchvision.transforms.functional import to_pil_image  # For correct image saving
from torchvision import transforms
from torch.utils.data import DataLoader
from models.networks import ResnetGenerator, NLayerDiscriminator

# Define dataset paths and batch sizes
datasets = {
    # "Rain100L": {
    #     "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/input",
    #     "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/target",
    #     "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain100L/results_8/",
    #     "batch_size": 8,  # Custom batch size per dataset
    #     "sample_size": 100  # Take only 100 samples
    # },
    "DID-MDN-Heavy": {
        "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/rainy",
        "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Heavy/non_rainy",
        "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_12/",
        "batch_size": 8,
        "sample_size": 200  # Take 200 samples
    }
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Medium/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Medium-rain/results_1/",
    # "batch_size": 8,
    # "sample_size": 200  # Take 200 samples
    # }
    # "DID-MDN-Medium": {
    # "input": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy",
    # "target": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/non_rainy",
    # "output": "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Light-rain/results_1/",
    # "batch_size": 6,
    # "sample_size": 200  # Take 200 samples
    # }
}


# ✅ Image transformations with fixed normalization
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)),
    transforms.ColorJitter(brightness=0.02, contrast=0.02, saturation=0.02),  # ✅ Subtle augmentation
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # ✅ Normalization for training
])

def denormalize(tensor):
    mean = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(tensor.device)
    std = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(tensor.device)
    tensor = tensor * std + mean
    return torch.clamp(tensor, 0.05, 0.95)  # ✅ Avoid clipping too aggressively
  # ✅ Ensures valid pixel values

# ✅ Initialize Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ✅ Initialize Model
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=128, ndf=256, device=device)

# ✅ Learning Rate Schedulers
scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=30, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=30, gamma=0.5)

# ✅ Training Configuration
num_epochs = 70
checkpoint_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/"
os.makedirs(checkpoint_dir, exist_ok=True)  # ✅ Ensure checkpoint folder exists


# ✅ Mixed Precision Training
scaler = torch.amp.GradScaler("cuda")
# start_epoch = load_latest_checkpoint(model, model.optimizer_G, model.optimizer_D, scheduler_G, scheduler_D, scaler, checkpoint_dir, device)
start_epoch = 0
print(f"🚀 Training will start from epoch {start_epoch}")

# ✅ Loop Through Each Dataset and Train
for dataset_name, paths in datasets.items():
    print(f"\n🔹 Training on {dataset_name} dataset...")

    input_dir = paths["input"]
    target_dir = paths["target"]
    output_dir = paths["output"]
    batch_size = paths["batch_size"]
    sample_size = paths["sample_size"]

    os.makedirs(output_dir, exist_ok=True)

    # ✅ Load dataset
    train_loader = DataLoader(
        RainDatasetSSL(
            dataset_name=dataset_name,
            input_dir=input_dir,
            target_dir=target_dir,
            transform=transform,
            sample_size=sample_size,
            output_dir=output_dir,
            save_masked=True
        ),
        batch_size=batch_size,
        shuffle=True
    )

    for epoch in range(start_epoch, num_epochs):
        for i, (masked_A, real_A, real_B, img_name) in enumerate(train_loader):
            masked_A, real_A, real_B = masked_A.to(device), real_A.to(device), real_B.to(device)

            # ✅ Optimize model and check loss
            loss_G, loss_D_A, loss_D_B = model.optimize_parameters(masked_A, real_B, scaler)

            # ✅ Ensure all loss values are valid tensors
            if not isinstance(loss_G, torch.Tensor) or not isinstance(loss_D_A, torch.Tensor) or not isinstance(loss_D_B, torch.Tensor):
                print(f"⚠️ Error: Loss values must be tensors! Received: loss_G={loss_G}, loss_D_A={loss_D_A}, loss_D_B={loss_D_B}")
                continue

            # ✅ Check for NaN values in loss
            if torch.isnan(loss_G).any() or torch.isnan(loss_D_A).any() or torch.isnan(loss_D_B).any():
                print("⚠️ NaN detected in loss! Skipping update")
                continue

            print(f"Epoch {epoch}/{num_epochs}, Batch {i}/{len(train_loader)}, Image: {img_name[0]}, "
                  f"Loss_G: {loss_G.item():.4f}, Loss_D_A: {loss_D_A.item():.4f}, Loss_D_B: {loss_D_B.item():.4f}")

        scheduler_G.step(loss_G)
        scheduler_D.step((loss_D_A + loss_D_B) * 0.8)

        torch.cuda.empty_cache()  # ✅ Prevent OOM errors

        # ✅ Save checkpoint every 5 epochs
        if epoch % 5 == 0 or epoch == num_epochs - 1:
            torch.save({
                'epoch': epoch,
                'model_G_A_state_dict': model.netG_A.state_dict(),
                'model_G_B_state_dict': model.netG_B.state_dict(),
                'model_D_A_state_dict': model.netD_A.state_dict(),
                'model_D_B_state_dict': model.netD_B.state_dict(),
                'optimizer_G_state_dict': model.optimizer_G.state_dict(),
                'optimizer_D_state_dict': model.optimizer_D.state_dict(),
                'scheduler_G_state_dict': scheduler_G.state_dict(),
                'scheduler_D_state_dict': scheduler_D.state_dict(),
                'scaler_state_dict': scaler.state_dict()
            }, os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch}.pth"))
            print(f"✅ Checkpoint saved at epoch {epoch}")

    # ✅ Save final generated images with correct colors
    if epoch == num_epochs - 1:
        model.netG_A.eval()
        with torch.no_grad():
            for i, (_, real_A, _, img_names) in enumerate(train_loader):
                real_A = real_A.to(device)
                fake_B = model.netG_A(real_A)

                # ✅ Apply Denormalization Properly
                fake_B = denormalize(fake_B)
                fake_B = torch.clamp(fake_B, 0.05, 0.95)  # ✅ Ensure correct color range

                for j in range(real_A.size(0)):
                    img_name = img_names[j]
                    save_path = os.path.join(output_dir, f"{dataset_name}_epoch_{epoch}_de_rained_{img_name}")

                    # ✅ Convert to PIL image correctly
                    pil_img = to_pil_image(fake_B[j].detach().cpu())
                    pil_img.save(save_path)

                    print(f"✅ Saved: {save_path}")

print("🚀 Training on all datasets completed successfully! ✅")


Using device: cuda


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🚀 Training will start from epoch 0

🔹 Training on DID-MDN-Heavy dataset...
Epoch 0/70, Batch 0/25, Image: rainy_1733.jpg, Loss_G: 48.6769, Loss_D_A: 1.0063, Loss_D_B: 1.1324
Epoch 0/70, Batch 1/25, Image: rainy_1591.jpg, Loss_G: 49.2385, Loss_D_A: 9.0522, Loss_D_B: 3.8404
Epoch 0/70, Batch 2/25, Image: rainy_1426.jpg, Loss_G: 46.0179, Loss_D_A: 0.5221, Loss_D_B: 1.5276
Epoch 0/70, Batch 3/25, Image: rainy_1771.jpg, Loss_G: 46.3460, Loss_D_A: 1.4291, Loss_D_B: 0.8378
Epoch 0/70, Batch 4/25, Image: rainy_2453.jpg, Loss_G: 46.3683, Loss_D_A: 0.4513, Loss_D_B: 0.4582
Epoch 0/70, Batch 5/25, Image: rainy_335.jpg, Loss_G: 45.1559, Loss_D_A: 0.4000, Loss_D_B: 0.3325
Epoch 0/70, Batch 6/25, Image: rainy_1269.jpg, Loss_G: 37.9159, Loss_D_A: 0.2223, Loss_D_B: 0.1181
Epoch 0/70, Batch 7/25, Image: rainy_104.jpg, Loss_G: 38.0816, Loss_D_A: 0.1191, Loss_D_B: 0.0893
Epoch 0/70, Batch 8/25, Image: rainy_3014.jpg, Loss_G: 32.5191, Loss_D_A: 0.1418, Loss_D_B: 0.1170
Epoch 0/70, Batch 9/25, Image: rainy

/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


✅ Checkpoint saved at epoch 0
Epoch 1/70, Batch 0/25, Image: rainy_1656.jpg, Loss_G: 17.0067, Loss_D_A: 0.0450, Loss_D_B: 0.0404
Epoch 1/70, Batch 1/25, Image: rainy_3193.jpg, Loss_G: 19.2944, Loss_D_A: 0.0502, Loss_D_B: 0.0453
Epoch 1/70, Batch 2/25, Image: rainy_2492.jpg, Loss_G: 20.8307, Loss_D_A: 0.0481, Loss_D_B: 0.0548
Epoch 1/70, Batch 3/25, Image: rainy_3164.jpg, Loss_G: 19.7460, Loss_D_A: 0.0354, Loss_D_B: 0.0565
Epoch 1/70, Batch 4/25, Image: rainy_773.jpg, Loss_G: 19.7659, Loss_D_A: 0.0428, Loss_D_B: 0.0540
Epoch 1/70, Batch 5/25, Image: rainy_662.jpg, Loss_G: 21.5215, Loss_D_A: 0.0427, Loss_D_B: 0.0586
Epoch 1/70, Batch 6/25, Image: rainy_1957.jpg, Loss_G: 22.8705, Loss_D_A: 0.0509, Loss_D_B: 0.0623
Epoch 1/70, Batch 7/25, Image: rainy_447.jpg, Loss_G: 21.8180, Loss_D_A: 0.0453, Loss_D_B: 0.0470
Epoch 1/70, Batch 8/25, Image: rainy_1901.jpg, Loss_G: 26.5928, Loss_D_A: 0.0687, Loss_D_B: 0.0386
Epoch 1/70, Batch 9/25, Image: rainy_3336.jpg, Loss_G: 20.4415, Loss_D_A: 0.0704, 

In [ ]:
final_checkpoint_path = os.path.join(checkpoint_dir, "final_trained_model.pth")

torch.save({
    'epoch': num_epochs,
    'model_G_A_state_dict': model.netG_A.state_dict(),
    'model_G_B_state_dict': model.netG_B.state_dict(),
    'model_D_A_state_dict': model.netD_A.state_dict(),
    'model_D_B_state_dict': model.netD_B.state_dict(),
    'optimizer_G_state_dict': model.optimizer_G.state_dict(),
    'optimizer_D_state_dict': model.optimizer_D.state_dict(),
    'scheduler_G_state_dict': scheduler_G.state_dict(),
    'scheduler_D_state_dict': scheduler_D.state_dict(),
    'scaler_state_dict': scaler.state_dict()
}, final_checkpoint_path)

print(f"✅ Final trained model saved at: {final_checkpoint_path}")


✅ Final trained model saved at: /content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth


In [6]:
import torch
# Load the model
# ✅ Define the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth", map_location=device)
model = DeRainCycleGANSSL(input_nc=3, output_nc=3, ngf=128, ndf=256, device=device)
scaler = torch.amp.GradScaler("cuda")
# ✅ Learning Rate Schedulers
scheduler_G = torch.optim.lr_scheduler.StepLR(model.optimizer_G, step_size=30, gamma=0.5)
scheduler_D = torch.optim.lr_scheduler.StepLR(model.optimizer_D, step_size=30, gamma=0.5)

model.netG_A.load_state_dict(checkpoint['model_G_A_state_dict'])
model.netG_B.load_state_dict(checkpoint['model_G_B_state_dict'])
model.netD_A.load_state_dict(checkpoint['model_D_A_state_dict'])
model.netD_B.load_state_dict(checkpoint['model_D_B_state_dict'])

# Load optimizers & scaler (if resuming training)
model.optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
model.optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
scheduler_G.load_state_dict(checkpoint['scheduler_G_state_dict'])
scheduler_D.load_state_dict(checkpoint['scheduler_D_state_dict'])
scaler.load_state_dict(checkpoint['scaler_state_dict'])

print(f"✅ Model loaded from: /content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth")


<ipython-input-6-096fa9f893d0>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trai

✅ Model loaded from: /content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth


In [ ]:
import torch
from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from models.networks import ResnetGenerator

# ✅ Load trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_G_A = ResnetGenerator(input_nc=3, output_nc=3, ngf=128, norm_layer=torch.nn.InstanceNorm2d, n_blocks=9).to(device)
checkpoint = torch.load("/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth", map_location=device)
model_G_A.load_state_dict(checkpoint['model_G_A_state_dict'])
model_G_A.eval()

# ✅ Define image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    # transforms.CenterCrop((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ✅ Load new rainy image
img_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy/rainy_88.jpg"
img = Image.open(img_path).convert("RGB")
img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension

# ✅ Perform inference
with torch.no_grad():
    de_rained_tensor = model_G_A(img_tensor)

# ✅ Convert back to an image
mean = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
std = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
de_rained_tensor = de_rained_tensor * std + mean  # Denormalization
de_rained_tensor = torch.clamp(de_rained_tensor, 0.05, 0.95)

# ✅ Save the de-rained image
de_rained_image = to_pil_image(de_rained_tensor.squeeze(0).cpu())
de_rained_image.save("/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain_Light_rainy_rainy_88.jpg")

print("✅ Inference complete! De-rained image saved.")


<ipython-input-26-4012adf4af5a>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_tr

✅ Inference complete! De-rained image saved.


In [ ]:
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
from PIL import Image
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np
from models.networks import ResnetGenerator

# ✅ Load trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_G_A = ResnetGenerator(input_nc=3, output_nc=3, ngf=128, norm_layer=torch.nn.InstanceNorm2d, n_blocks=9).to(device)
checkpoint = torch.load("/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth", map_location=device, weights_only=True)
model_G_A.load_state_dict(checkpoint['model_G_A_state_dict'])
model_G_A.eval()

# ✅ Define image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# ✅ Load new rainy image
img_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/rainy/rainy_88.jpg"
gt_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Rain_Light/non_rainy/non_rainy_88.jpg"  # Ground truth

img = Image.open(img_path).convert("RGB")
gt = Image.open(gt_path).convert("RGB")

img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension
gt_tensor = transform(gt).unsqueeze(0).to(device)

# ✅ Perform inference
with torch.no_grad():
    de_rained_tensor = model_G_A(img_tensor)

# ✅ Convert back to an image
mean = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
std = torch.tensor([0.5, 0.5, 0.5]).view(3, 1, 1).to(device)
de_rained_tensor = de_rained_tensor * std + mean  # Denormalization
de_rained_tensor = torch.clamp(de_rained_tensor, 0.05, 0.95)

gt_np = np.array(img.resize((256, 256)), dtype=np.float32) / 255.0  # Normalize GT to 0-1
de_rained_np = np.array(to_pil_image(de_rained_tensor.squeeze(0).cpu()), dtype=np.float32) / 255.0  # Normalize output

# ✅ Compute PSNR & SSIM properly
psnr_value = psnr(gt_np, de_rained_np, data_range=1.0)  # Use 1.0 as max range
ssim_value = ssim(gt_np, de_rained_np, data_range=1.0, channel_axis=-1, win_size=3)

# ✅ Save the de-rained image
de_rained_image = to_pil_image(de_rained_tensor.squeeze(0).cpu())
de_rained_image.save("/content/drive/MyDrive/Khabeer - IRP/Dataset/Rain_Light_rainy_rainy_882.jpg")

print(f"✅ Inference complete! De-rained image saved.")
print(f"📌 PSNR: {psnr_value:.4f}, SSIM: {ssim_value:.4f}")


✅ Inference complete! De-rained image saved.
📌 PSNR: 19.1212, SSIM: 0.5683


In [ ]:
pip install scikit-image


In [8]:
pip install lpips

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.4 MB/s eta 0:00:00


In [12]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import lpips  # LPIPS Perceptual Loss
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import math
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import numpy as np

# ------------------------------
# 🔹 Dataset Handling
# ------------------------------
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.image_filenames[idx]

# ------------------------------
# 🔹 Image Transformations (Ensuring Proper Normalization)
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize to [-1,1] for LPIPS
])

# ------------------------------
# 🔹 Improved Autoencoder
# ------------------------------
class Autoencoder(nn.Module):
    def __init__(self, input_nc):
        super(Autoencoder, self).__init__()

        # 🔹 Encoder
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder4 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))

        # 🔹 Bottleneck
        self.bottleneck = nn.Sequential(nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))

        # 🔹 Decoder
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(512 + 512, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder5 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1),nn.Sigmoid())  # ✅ Changed to Tanh for [-1,1] range

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        b = self.bottleneck(e4)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e4], dim=1))
        d3 = self.decoder3(torch.cat([d2, e3], dim=1))
        d4 = self.decoder4(torch.cat([d3, e2], dim=1))
        d5 = self.decoder5(torch.cat([d4, e1], dim=1))
        return d5

# ------------------------------
# 🔹 Perceptual Loss (LPIPS) + SSIM Loss
# ------------------------------
class SSIMLoss(nn.Module):
    def __init__(self):
        super(SSIMLoss, self).__init__()

    def forward(self, img1, img2):
        return 1 - ssim(img1.cpu().detach().numpy().transpose(1, 2, 0), img2.cpu().detach().numpy().transpose(1, 2, 0), data_range=2.0, channel_axis=2)

# ------------------------------
# 🔹 Training Configuration
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(input_nc=3).to(device)
mse_loss = nn.MSELoss()
ssim_loss = SSIMLoss()
lpips_loss_fn = lpips.LPIPS(net='alex').to(device)  # ✅ LPIPS Perceptual Loss

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0002)

# ✅ Load dataset
dataset_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_12/"
train_dataset = ImageDataset(dataset_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# ------------------------------
# 🔹 Training Loop
# ------------------------------
output_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/final_reconstructed_images_12_7/"
os.makedirs(output_dir, exist_ok=True)
# Convert output from [-1,1] to [0,1]
def denormalize_image(tensor):
    return (tensor + 1) / 2  # ✅ Convert from [-1,1] → [0,1]

epochs = 60
for epoch in range(epochs):
    total_psnr, total_ssim, total_lpips, num_images = 0, 0, 0, 0

    for images, filenames in train_loader:
        images = images.to(device)
        reconstructed = autoencoder(images)

        mse_loss_value = mse_loss(reconstructed, images)
        ssim_loss_value = torch.tensor([
            ssim(
                reconstructed[i].cpu().detach().numpy().transpose(1, 2, 0),
                images[i].cpu().detach().numpy().transpose(1, 2, 0),
                data_range=2.0,  # 🔹 FIX: Explicit data_range for SSIM
                channel_axis=2
            ) for i in range(images.size(0))
        ]).mean()
        lpips_loss_value = lpips_loss_fn(images, reconstructed).mean()

        #  Loss
        # ✅ Reduce LPIPS loss contribution to avoid dark artifacts
        total_loss = 0.8 * mse_loss_value + 0.2 * ssim_loss_value + 0.02 * lpips_loss_value

        optimizer.zero_grad()
        total_loss.backward()
        nn.utils.clip_grad_norm_(autoencoder.parameters(), max_norm=5)  # ✅ Gradient Clipping
        optimizer.step()

        total_psnr += psnr(images.cpu().detach().numpy(), reconstructed.cpu().detach().numpy(), data_range=2.0)
        total_ssim += ssim_loss_value.item()
        total_lpips += lpips_loss_value.item()
        num_images += 1

         # ✅ Save Images in the Final Epoch
        if epoch == epochs - 1:
            for i in range(images.size(0)):
                save_path = os.path.join(output_dir, f"reconstructed_{filenames[i]}")
                # Denormalize output before saving
                reconstructed = denormalize_image(reconstructed)
                save_image(reconstructed[i], save_path)


    print(f"Epoch [{epoch+1}/{epochs}], PSNR: {total_psnr/num_images:.4f}, SSIM: {total_ssim/num_images:.4f}, LPIPS: {total_lpips/num_images:.4f}")

print("\n🎯 Autoencoder Training & Evaluation Completed! ✅")
print(f"✅ Final reconstructed images saved to: {output_dir}")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
Epoch [1/60], PSNR: 11.4478, SSIM: 0.0555, LPIPS: 0.9069
Epoch [2/60], PSNR: 13.0224, SSIM: 0.1164, LPIPS: 0.8189
Epoch [3/60], PSNR: 13.2814, SSIM: 0.1369, LPIPS: 0.7188
Epoch [4/60], PSNR: 13.4929, SSIM: 0.1616, LPIPS: 0.6192
Epoch [5/60], PSNR: 13.6207, SSIM: 0.1911, LPIPS: 0.5724
Epoch [6/60], PSNR: 13.7125, SSIM: 0.2222, LPIPS: 0.5395
Epoch [7/60], PSNR: 13.7478, SSIM: 0.2415, LPIPS: 0.5159
Epoch [8/60], PSNR: 13.7969, SSIM: 0.2568, LPIPS: 0.4991
Epoch [9/60], PSNR: 13.8208, SSIM: 0.2677, LPIPS: 0.4864
Epoch [10/60], PSNR: 13.8356, SSIM: 0.2757, LPIPS: 0.4755
Epoch [11/60], PSNR: 13.8224, SSIM: 0.2825, LPIPS: 0.4659
Epoch [12/60], PSNR: 13.8367, SSIM: 0.2883, LPIPS: 0.4562
Epoch [13/60], PSNR: 13.8550, SSIM: 0.2933, LPIPS: 0.4459
Epoch [14/60], PSNR: 13.8780, SSIM: 0.2980, LPIPS: 0.4387
Epoch [15/60], PSNR: 13.8803, SSIM: 0

In [14]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import lpips  # LPIPS Perceptual Loss
from torchvision.utils import save_image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# ------------------------------
# 🔹 Dataset Handling
# ------------------------------
class ImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = sorted([f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.image_filenames[idx]

# ------------------------------
# 🔹 Image Transformations (Ensuring Proper Normalization)
# ------------------------------
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# ------------------------------
# 🔹 Improved Autoencoder (Larger Bottleneck)
# ------------------------------
class Autoencoder(nn.Module):
    def __init__(self, input_nc):
        super(Autoencoder, self).__init__()

        # 🔹 Encoder
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder4 = nn.Sequential(nn.Conv2d(256, 1024, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))  # 🔹 Larger bottleneck

        # 🔹 Bottleneck (Expanded)
        self.bottleneck = nn.Sequential(nn.Conv2d(1024, 1024, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))

        # 🔹 Decoder
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(1024, 1024, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(1024 + 1024, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder5 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1), nn.Sigmoid())  # 🔹 Sigmoid for [0,1] range

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        b = self.bottleneck(e4)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e4], dim=1))
        d3 = self.decoder3(torch.cat([d2, e3], dim=1))
        d4 = self.decoder4(torch.cat([d3, e2], dim=1))
        d5 = self.decoder5(torch.cat([d4, e1], dim=1))
        return d5

# ------------------------------
# 🔹 Training Configuration
# ------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = Autoencoder(input_nc=3).to(device)
mse_loss = nn.MSELoss()
lpips_loss_fn = lpips.LPIPS(net='alex').to(device)  # ✅ LPIPS Perceptual Loss

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0002)

# ✅ Load dataset
dataset_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/results_12/"
train_dataset = ImageDataset(dataset_path, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# ✅ Output Directory
output_dir = "/content/drive/MyDrive/Khabeer - IRP/Dataset/DID-MDN-datasets/DID-MDN-training-split/Heavy-rain/final_reconstructed_images_12_10/"
os.makedirs(output_dir, exist_ok=True)

# ------------------------------
# 🔹 Training Loop
# ------------------------------
def compute_ssim(img1, img2):
    return ssim(img1.cpu().detach().numpy().transpose(1, 2, 0), img2.cpu().detach().numpy().transpose(1, 2, 0), data_range=1.0, channel_axis=2)

epochs = 80  # 🔹 More epochs for stability
for epoch in range(epochs):
    total_psnr, total_ssim, total_lpips, num_images = 0, 0, 0, 0

    for images, filenames in train_loader:
        images = images.to(device)
        reconstructed = autoencoder(images)

        mse_loss_value = mse_loss(reconstructed, images)
        ssim_loss_value = torch.tensor([compute_ssim(reconstructed[i], images[i]) for i in range(images.size(0))]).mean()
        lpips_loss_value = lpips_loss_fn(images, reconstructed).mean()

        # ✅ Adjusted Loss Function Weights (Better SSIM)
        total_loss = 0.5 * mse_loss_value + 0.4 * (1 - ssim_loss_value) + 0.05 * lpips_loss_value  # 🔹 Higher SSIM weight

        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        total_psnr += psnr(images.cpu().detach().numpy(), reconstructed.cpu().detach().numpy(), data_range=1.0)
        total_ssim += ssim_loss_value.item()
        total_lpips += lpips_loss_value.item()
        num_images += 1

        # ✅ Save Images in the Final Epoch
        if epoch == epochs - 1:
            for i in range(images.size(0)):
                save_path = os.path.join(output_dir, f"reconstructed_{filenames[i]}")
                save_image(reconstructed[i], save_path)

    print(f"Epoch [{epoch+1}/{epochs}], PSNR: {total_psnr/num_images:.4f}, SSIM: {total_ssim/num_images:.4f}, LPIPS: {total_lpips/num_images:.4f}")

# ✅ Save the trained model
torch.save(autoencoder.state_dict(), "/content/drive/MyDrive/Khabeer - IRP/Dataset/autoencoder_trained.pth")

print("\n✅ Model saved as autoencoder_trained.pth ✅")


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth
Epoch [1/80], PSNR: 11.7606, SSIM: 0.0962, LPIPS: 0.5727
Epoch [2/80], PSNR: 15.8081, SSIM: 0.1876, LPIPS: 0.4248
Epoch [3/80], PSNR: 19.0521, SSIM: 0.3352, LPIPS: 0.3540
Epoch [4/80], PSNR: 20.8190, SSIM: 0.4239, LPIPS: 0.2853
Epoch [5/80], PSNR: 22.0705, SSIM: 0.5070, LPIPS: 0.2332
Epoch [6/80], PSNR: 22.9662, SSIM: 0.5808, LPIPS: 0.1988
Epoch [7/80], PSNR: 23.7724, SSIM: 0.6382, LPIPS: 0.1692
Epoch [8/80], PSNR: 24.5388, SSIM: 0.6883, LPIPS: 0.1414
Epoch [9/80], PSNR: 25.1847, SSIM: 0.7231, LPIPS: 0.1153
Epoch [10/80], PSNR: 25.9672, SSIM: 0.7516, LPIPS: 0.0939
Epoch [11/80], PSNR: 26.3819, SSIM: 0.7735, LPIPS: 0.0814
Epoch [12/80], PSNR: 26.9687, SSIM: 0.7907, LPIPS: 0.0727
Epoch [13/80], PSNR: 27.1899, SSIM: 0.8042, LPIPS: 0.0678
Epoch [14/80], PSNR: 27.5502, SSIM: 0.8142, LPIPS: 0.0637
Epoch [15/80], PSNR: 27.8129, SSIM: 0

In [ ]:
# ✅ Define a path to save the model
autoencoder_save_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/autoencoder_model.pth"

# ✅ Save function after training completes
torch.save({
    'epoch': epochs,
    'model_state_dict': autoencoder.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, autoencoder_save_path)

print(f"✅ Autoencoder model saved at: {autoencoder_save_path}")


In [ ]:
# ✅ Load function
checkpoint = torch.load(autoencoder_save_path, map_location=device)

# ✅ Load model and optimizer states
autoencoder.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
start_epoch = checkpoint['epoch']

# ✅ Set model to evaluation mode for inference
autoencoder.eval()
print(f"✅ Autoencoder model loaded from epoch {start_epoch}")


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import lpips  # LPIPS Perceptual Loss
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from models.networks import ResnetGenerator  # ✅ GAN Model

# ✅ Define image transformations (ensure consistency across models)
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize to [-1,1]
])

# ✅ Function to denormalize and convert tensor to image
def tensor_to_pil(image_tensor):
    image_tensor = image_tensor * 0.5 + 0.5  # Denormalize
    image_tensor = torch.clamp(image_tensor, 0, 1)  # Clip values
    image_np = image_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)  # Convert to NumPy
    return Image.fromarray((image_np * 255).astype(np.uint8))

# ✅ Load GAN Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_G_A = ResnetGenerator(input_nc=3, output_nc=3, ngf=128, norm_layer=torch.nn.InstanceNorm2d, n_blocks=9).to(device)

# ✅ Load GAN Weights
gan_checkpoint_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/checkpoint_1/final_trained_model.pth"
checkpoint = torch.load(gan_checkpoint_path, map_location=device)
model_G_A.load_state_dict(checkpoint['model_G_A_state_dict'])
model_G_A.eval()
print("✅ GAN Model Loaded!")

# ✅ Define Autoencoder Model
class Autoencoder(nn.Module):
    def __init__(self, input_nc):
        super(Autoencoder, self).__init__()
        self.encoder1 = nn.Sequential(nn.Conv2d(input_nc, 64, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder2 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder3 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.encoder4 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.bottleneck = nn.Sequential(nn.Conv2d(512, 512, kernel_size=4, stride=2, padding=1), nn.LeakyReLU(0.2, True))
        self.decoder1 = nn.Sequential(nn.ConvTranspose2d(512, 512, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder2 = nn.Sequential(nn.ConvTranspose2d(512 + 512, 256, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder3 = nn.Sequential(nn.ConvTranspose2d(256 + 256, 128, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder4 = nn.Sequential(nn.ConvTranspose2d(128 + 128, 64, kernel_size=4, stride=2, padding=1), nn.ReLU(True))
        self.decoder5 = nn.Sequential(nn.ConvTranspose2d(64 + 64, input_nc, kernel_size=4, stride=2, padding=1),nn.Sigmoid())  # ✅ Changed to Tanh for [-1,1] range

    def forward(self, x):
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        b = self.bottleneck(e4)
        d1 = self.decoder1(b)
        d2 = self.decoder2(torch.cat([d1, e4], dim=1))
        d3 = self.decoder3(torch.cat([d2, e3], dim=1))
        d4 = self.decoder4(torch.cat([d3, e2], dim=1))
        d5 = self.decoder5(torch.cat([d4, e1], dim=1))
        return d5

# ✅ Load Autoencoder Model
autoencoder = Autoencoder(input_nc=3).to(device)
autoencoder_checkpoint_path = "/content/drive/MyDrive/Khabeer - IRP/Dataset/autoencoder_model.pth"
checkpoint = torch.load(autoencoder_checkpoint_path, map_location=device)
autoencoder.load_state_dict(checkpoint['model_state_dict'])
autoencoder.eval()
print("✅ Autoencoder Model Loaded!")

# ✅ Initialize Loss Functions
lpips_loss_fn = lpips.LPIPS(net='alex').to(device)  # Perceptual Loss

# ✅ Compute Loss Metrics
def compute_metrics(gt, output):
    gt_np = gt.squeeze().cpu().numpy().transpose(1, 2, 0)
    output_np = output.squeeze().cpu().numpy().transpose(1, 2, 0)

    psnr_value = psnr(gt_np, output_np, data_range=1.0)
    ssim_value = ssim(gt_np, output_np, data_range=1.0, channel_axis=2)
    lpips_value = lpips_loss_fn(gt, output).mean().item()

    return psnr_value, ssim_value, lpips_value

def derain_pipeline(image_path, output_path):
    # ✅ Load and preprocess image
    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0).to(device)

    # ✅ GAN Model Inference
    with torch.no_grad():
        gan_output = model_G_A(img_tensor)

    # ✅ Autoencoder Inference
    with torch.no_grad():
        refined_output = autoencoder(gan_output)

    # ✅ Convert to PIL Image
    refined_pil = tensor_to_pil(refined_output)

    # ✅ Save Image
    refined_pil.save(output_path)

    # ✅ Compute Loss Scores
    psnr_value, ssim_value, lpips_value = compute_metrics(img_tensor, refined_output)

    print(f"✅ Inference Complete!")
    print(f"📌 PSNR: {psnr_value:.4f}, SSIM: {ssim_value:.4f}, LPIPS: {lpips_value:.4f}")

    return refined_pil, psnr_value, ssim_value, lpips_value



In [ ]:
input_image = "/content/drive/MyDrive/Khabeer - IRP/Dataset/test_rainy.jpg"
output_image = "/content/drive/MyDrive/Khabeer - IRP/Dataset/de-rained_test.jpg"

final_image, psnr_score, ssim_score, lpips_score = derain_pipeline(input_image, output_image)
